In [ ]:
import sys
import os
import numpy as np
import glob
from tifffile import imread, imwrite
import napari
from pathlib import Path
from skimage.measure import  label
import pandas as pd
from skimage import measure
from skimage.morphology import remove_small_objects

In [ ]:
Imagedir = '/Users/aimachine/fmi-assignment/Denoised/'
Resultdir = '/Users/aimachine/fmi-assignment/Results/'
Path(Resultdir).mkdir(exist_ok=True)


In [ ]:
#Use the probability image coming out of Ilastik for the (first) spot channel 
#and threshold to remove low probability detections
ProbabilityThreshold = 0.5
#Remove spots below a certain size, to get rid of tiney dots
min_size = 20

In [ ]:
Raw_path = os.path.join(Imagedir, '*tif')
filesRaw = glob.glob(Raw_path)
filesRaw.sort

In [ ]:
AllCount = []
AllName = []
for fname in filesRaw:
    Name = os.path.basename(os.path.splitext(fname)[0])
    TwoChannel = imread(fname)
    SpotChannel = TwoChannel[:,0,:,:]
    Binary = SpotChannel > ProbabilityThreshold
    Binary = remove_small_objects(Binary, min_size = min_size)
    Integer = label(Binary)
    waterproperties = measure.regionprops(Integer, Integer)
    labels = []
    for prop in waterproperties:
        if prop.label > 0:
                 
                  labels.append(prop.label)
    count = len(labels)
    imwrite(Resultdir + Name + '.tif', Integer.astype('uint16')) 
    AllName.append(Name)
    Allcount.append(count)
    
df = pd.DataFrame(list(zip(AllCount)), index =AllName, 
                                              columns =['Count'])

df.to_csv(Resultdir + '/' + CountMasks +  '.csv')  
df    
    